In [1]:
import pandas as pd
import spacy
from pyserini.index import IndexReader
from tqdm import tqdm, trange
import sys
import numpy as np

nlp = spacy.load("en_core_web_sm")

food = pd.read_csv('foodresults_new.csv')
ids = [i for i in range(len(food))]
food['recipe_id'] = ids
food.head()

,name,ingredients,recipe,rate,views,description,keywords,calories,fatContent,proteinContent,url,recipe_id
0,Chinese Fried Rice,"3/4 cup finely chopped onion,2 1/2 tablesp...",Heat 1 tbsp oil in wok; add chopped onions and...,5.0,541,This Chinese fried rice has the flavor those o...,"Pork,Poultry,Rice,Meat,Chinese,Asian,Low Chole...",497.8,16.3,22.3,https://www.food.com/recipe/chinese-fried-rice...,0
1,Chicken Noodle Soup,"2 teaspoons butter,1 cup sliced celer...","Melt butter in large pot.,Saut&eacute; the cel...",5.0,456,Make and share this Chicken Noodle Soup recipe...,"Chicken,Poultry,Meat,Low Cholesterol,Healthy,K...",304.8,8.0,24.1,https://www.food.com/recipe/chicken-noodle-sou...,1
2,Restaurant Teriyaki Sauce,"1/4 cup soy sauce,1 cup water,1/2 te...",Mix all but cornstarch and 1/4c water in a sau...,5.0,341,"You know the nice, sweet, thick Teriyaki sauce...","Low Protein,Low Cholesterol,Healthy,< 15 Mins,...",290.5,0.1,5.3,https://www.food.com/recipe/restaurant-teriyak...,2
3,Award Winning Chili,1 lb ground beef or 1 lb ground turk...,In a medium Dutch oven over medium-high heat a...,5.0,447,My husband and I created this recipe together....,"Beans,Meat,Low Cholesterol,Healthy,< 4 Hours,E...",300.6,7.2,27.0,https://www.food.com/recipe/award-winning-chil...,3
4,Ww 0 Point Weight Watchers Cabbage Soup,3 cups nonfat vegetable broth (beef is the...,Spray pot with non stick cooking spray saute o...,5.0,371,Make and share this Ww 0 Point Weight Watchers...,"Onions,Vegetable,Low Protein,Low Cholesterol,H...",22.0,0.1,1.0,https://www.food.com/recipe/ww-0-point-weight-...,4


In [2]:
import csv
import json
 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    collection = []
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        cnt = 0
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            dictionary = {}
            dictionary['id'] = cnt
            dictionary['contents'] = rows['name'] +' '+ rows['ingredients'] +' '+ rows['recipe']
            collection.append(dictionary)
            cnt += 1
     # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(collection, indent=4))
         
# Driver Code
 
# Decide the two file paths 
csvFilePath = r'foodresults_new.csv'
jsonFilePath = r'foodresults_new.json'
 
# Call the make_json function
make_json(csvFilePath, jsonFilePath)

In [5]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator  -threads 1 -input ./foodresults_new -index indexes/foodresults_new -storePositions -storeDocvectors -storeContents

2021-12-01 21:30:23,741 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2021-12-01 21:30:23,742 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2021-12-01 21:30:23,742 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2021-12-01 21:30:23,742 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: ./foodresults_new
2021-12-01 21:30:23,742 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2021-12-01 21:30:23,742 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2021-12-01 21:30:23,743 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 1
2021-12-01 21:30:23,743 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Stemmer: porter
2021-12-01 21:30:23,743 INFO  [main] index.IndexCollection (I

In [7]:
from pyserini.index import IndexReader
index_reader = IndexReader('indexes/foodresults_new')

In [8]:
import itertools
import math
from pyserini.index import IndexReader
from tqdm import tqdm

# term_df: document frequency
# doc_vdc: term frequency in a given document
def BM25(query, doc_name, term_df, doc_vec, avg_dl):
    # parameters
    k1 = 1.2
    k3 = 1.2            
    b = 0.75
    
    analyzed = index_reader.analyze(query)
    document_length = 0
    for key in doc_vec[doc_name]:
        if key != None and key in doc_vec[doc_name] and doc_vec[doc_name][key] != None:
            document_length += int(doc_vec[doc_name][key]) * len(key)

    f = 0
    for term in set(analyzed):
        if term not in doc_vec[doc_name]:
            continue
        # term doc counts
        ctD = doc_vec[doc_name][term]
        # term query freq
        ctQ = analyzed.count(term)
        # document freq
        df = term_df[term]
        term_score = math.log((num_documents - df + 0.5)/(df + 0.5)) * ((k1 + 1)*(ctD))/(k1*(1-b+b*document_length/avg_dl) + ctD) * (k3 + 1) * ctQ / (k3 + ctQ)
        f += term_score
    return f

In [9]:
# store all the information needed
# class info
# document frequency
num_documents = index_reader.stats()['documents']
term_df = {}
# avg_dl = index_reader.stats()['total_terms'] / num_documents
avg_dl = 0
for term in index_reader.terms():
    term_df[term.term] = term.df
    avg_dl += len(term.term)*term.cf

avg_dl = avg_dl / num_documents

# c(w,d)
doc_vec = {}
doc_index = {}
for i in tqdm(range(num_documents)):
    doc_id = index_reader.convert_internal_docid_to_collection_docid(i)
    doc_index[i] = doc_id
    try:
        dv = index_reader.get_document_vector(doc_id)
        doc_vec[doc_id] = dv
    except:
        pass


100%|███████████████████████████████████████████████████████████████████████████| 57125/57125 [02:32<00:00, 374.47it/s]


In [11]:
queries = pd.read_excel('queries.xlsx')
queries

,query_id,content
0,1,tomato
1,2,egg
2,3,rice
3,4,chicken
4,5,beef
5,6,sugar
6,7,steak
7,8,apple
8,9,orange
9,10,lettuce


In [14]:
# query = 'tomato'
# query_term = query.split()
with open('bm25result_new.csv', 'w') as f:    
    f.write('query,recipe_id,recipe_name,url\n')
    # calculate the score and rank each document
    for i in tqdm(range(len(queries))):
        query = queries.loc[i]['content']
        score_dict = {}
        for j in range(num_documents):
            # try:
            docid = doc_index[j]
            if str(docid) in doc_vec:
                score_dict[str(docid)] = BM25(query, str(docid), term_df, doc_vec, avg_dl)

        rank = sorted(score_dict.items(), key=lambda item: item[1], reverse=True)[:110]


        for ids, score in rank:
            f.write("%s,%s,%s,%s\n" %(query, ids, food.loc[int(ids)]['name'].replace(',',' '), food.loc[int(ids)]['url']))

f.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [02:26<00:00,  4.88s/it]


In [13]:
test = pd.read_csv('bm25result_new.csv', encoding='cp1252')
test

,query,recipe_id,recipe_name,url
0,tomato,6479,Macaroni and Tomatoes,https://www.food.com/recipe/macaroni-and-tomat...
1,tomato,50070,Tofu Stuffed Cherry Tomatoes,https://www.food.com/recipe/tofu-stuffed-cherr...
2,tomato,23847,Herb Grilled Tomatoes,https://www.food.com/recipe/herb-grilled-tomat...
3,tomato,54385,Canned Tomatoes (Vine Ripened Tomatoes in Toma...,https://www.food.com/recipe/canned-tomatoes-vi...
4,tomato,50637,Italian Tomato Sauce,https://www.food.com/recipe/italian-tomato-sau...
...,...,...,...,...
3295,ice-cream,22515,Brown Cow (Root Beer Shake),https://www.food.com/recipe/brown-cow-root-bee...
3296,ice-cream,29449,Classic Jello Milkshake,https://www.food.com/recipe/classic-jello-milk...
3297,ice-cream,26987,Barney Blaster,https://www.food.com/recipe/barney-blaster-213530
3298,ice-cream,18782,Strawberry Float,https://www.food.com/recipe/strawberry-float-2...


In [10]:
# use merge to join 2 datasets

'Macaroni and Tomatoes'